## Configuracion

In [0]:
import yaml
import time
from pathlib import Path
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput, 
    ServedModelInput,
    ServedModelInputWorkloadSize,
    EndpointStateReady
)
from config_loader import Config

paths = Config.get_paths()
params = Config.get_params()

model_short_name = params['model_params']['model_name']
model_full_name = f"{paths['catalog']}.{paths['schemas']['feature_store']}.{model_short_name}"
model_alias = params['model_params']['alias']

endpoint_name = params['serving_params']['endpoint_name']
# Convertimos el string del YAML al Enum del SDK
workload_size = ServedModelInputWorkloadSize[params['serving_params']['workload_size']]
scale_to_zero = params['serving_params']['scale_to_zero']

## Obtener Version del Modelo

In [0]:
w = WorkspaceClient()

try:
    model_details = w.model_versions.get_by_alias(model_full_name, alias=model_alias)
    version_to_deploy = model_details.version
    print(f"📦 Detectada versión {version_to_deploy} para el alias '{model_alias}'")
except Exception as e:
    print(f"❌ Error: No se encontró el alias '{model_alias}' en el modelo {model_full_name}")
    raise e

## Configuracion del endpoint

In [0]:
serving_config = EndpointCoreConfigInput(
    served_models=[
        ServedModelInput(
            model_name=model_full_name,
            model_version=version_to_deploy,
            workload_size=workload_size,
            scale_to_zero_enabled=scale_to_zero
        )
    ]
)

## Crear o actualizar endpoint

In [0]:
existing_endpoints = [e.name for e in w.serving_endpoints.list()]

if endpoint_name in existing_endpoints:
    print(f"🔄 Actualizando endpoint '{endpoint_name}' a la versión {version_to_deploy}")
    w.serving_endpoints.update_config(name=endpoint_name, served_models=serving_config.served_models)
else:
    print(f"🚀 Creando nuevo endpoint '{endpoint_name}'")
    w.serving_endpoints.create(name=endpoint_name, config=serving_config)


print("⏳ Esperando a que el endpoint esté listo (esto puede tardar unos minutos)...")

while True:
    endpoint = w.serving_endpoints.get(name=endpoint_name)
    status = endpoint.state.ready
    
    print(f"   Estado actual: {status}")
    
    if status == EndpointStateReady.READY:
        print(f"✅ ¡Éxito! Endpoint '{endpoint_name}' activo.")
        break
    elif status == EndpointStateReady.NOT_READY:
        print("⚠️ El endpoint reporta estado NOT_READY. Revisa los logs en el UI.")
        break
    else:
        time.sleep(20)